In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 이거 실행하고 다시 런타임 시작
# 한글 폰트
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 4 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 2s (5,149 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

# 라이브러리 & 데이터 불러오기

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from tqdm import tqdm
import warnings
import plotly.express as px
from matplotlib.colors import LinearSegmentedColormap
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, StratifiedKFold
sns.set(font='NaNumBarunGothic')



%matplotlib inline


pd.options.display.max_columns = None # 모든 컬럼 볼 수 있게끔
# 경고 끄기
warnings.filterwarnings(action='ignore')

In [3]:
# 데이터 불러오기

# 기업개요 불러오기
active = pd.read_excel('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/액티브 중소법인 재무보유.xlsx')
# active는 컬럼명이 NM1 까지 붙어있어서 concat이 이상하게 된다. 그래서 컬럼명 변경~
active.rename(columns = {'CMP_NM1':'CMP_NM'}, inplace=True)

active2 = pd.read_excel('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/액티브 중소법인 재무보유.xlsx', sheet_name = '기업개요')

closing = pd.read_excel('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/휴폐업 중소법인 재무보유.xlsx')

# 휴폐업이력 불러오기
active_important = pd.read_excel('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/액티브 중소법인 재무보유.xlsx',sheet_name = '휴폐업이력')
closing_important = pd.read_excel('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/휴폐업 중소법인 재무보유.xlsx',sheet_name = '휴폐업이력')

# 재무제표 불러오기
fs = pd.read_table('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/재무데이터.txt', encoding = 'cp949')



# 기존 데이터에서 feature selection

In [4]:


def make_preprocessing(active, closing, active_important, closing_important):

  active['휴폐업_여부']=[0]*len(active)
  closing['휴폐업_여부'] = [1]*len(closing)
  df = pd.concat([active, closing], axis=0)

  df['CMP_PFIX_NM'].fillna('', inplace=True)
  df['CMP_SFIX_NM'].fillna('', inplace=True)

  # CMP_PFIX_NM, CMP_NM, CMP_SFIX_NM, CMP_ENM 데이터 가공처리
  # (주)가 있는지 없는지 여부만 판단하는 컬럼으로 합쳐준다.
  # CMP_PFIX_NM, CMP_SFIX_NM은 drop ! 
  # CMP_ENM(기업영문명) drop
  df['tmp'] = df['CMP_PFIX_NM'] + df['CMP_SFIX_NM']
  df['(주)여부'] = df.apply(lambda x: 1 if '(주)' in x['tmp'] else 0, axis=1)

  df.drop(['CMP_PFIX_NM', 'CMP_SFIX_NM', 'CMP_ENM', 'tmp'], axis=1, inplace=True)


  # 공기업구분여부도 모델에 큰영향을 줄 것 같지 않으므로 drop한다. - 액티브든, 휴폐업이든 공기업인 기업이 없다.
  df.drop('PBCO_GB', axis=1,inplace=True)

  # 개인법인구분에서 모드 법인으로 나오므로 이것도 컬럼 삭제한다. - 분류모델에 유의미한 영향을 주지 못한다.
  df.drop('PSN_CORP_GB', axis=1, inplace=True)

  
  # 본점지점구분은 그래도 좀 나오니깐 일단 컬럼으로 넣어보고 성능에 유의미한 영향을 주지않으면 빼자
  # 국외투자법인여부는 나름 유의미하게 분포되어 있는 것 같다. 효과는 미비할 것 같지만 그나마 쓸만할 것 같아서 빼지않는다.
  # 벤처기업여부 또한 나름 유의미하게 분포.

  # 상장코드는 있는게 매우 드물다.
  # 상장코드 - 있으면 1, 없으면 0으로 채워넣자
  df['LIST_CD'].fillna(0, inplace=True)
  df['LIST_CD'] = df['LIST_CD'].apply(lambda x: 1 if x!=0 else 0)


  # 산업코드차수 전체에서 10개 빼고 모두 '10'으로 라벨링 돼있다. 그냥 drop
  df.drop('IND_CD_ORDR', axis=1, inplace = True)


  # 산업코드 보류하자 이거 진짜 어렵다...............
  # 산업코드1, 2, 3
  # 일단 null 값은 etc로 통합 - 12개정도밖에 없다.
  # 산업코드 1에서 10개 이하로 나오는 산업코드는 etc로 통합해준다. -> 나중에 분류 모델링의 성능평가를 위해
  # 아니면 이것도 count값을 넣어줄까
  df['IND_CD1'].fillna('etc', inplace=True)
  change_index= []
  for i in range(len(df['IND_CD1'].value_counts())):
    if df['IND_CD1'].value_counts().values[i]<=10:
      change_index.append(df['IND_CD1'].value_counts().index[i])
  df['IND_CD1'].replace(change_index, 'etc', inplace=True)


  # 산업코드 2, 3은 널값이 많으니까 drop
  df.drop(['IND_CD2','IND_CD3'], axis=1, inplace=True)

  # 공공기관 유형
  # 값의 차이가 너무 미세하다.
  # drop
  df.drop('PB_ORG_TYP', axis=1, inplace=True)


  # 중견기업보호여부 일단 넣자. 막 유의미해보이진 않는데 그래도
  df['MDSCO_PRTC_YN'].fillna('N', inplace=True)


  # 본점기업코드
  # 널값은 etc로 채워주고
  # 본점기업코드는 value_counts의 개수를 대신 넣어주면 좋을 것같다. 
  # 숫자가 높을수록 체인점을 많이 보유한 업종임을 판단할 수 있다.
  # 시간이 너무 오래걸린다.
  df['HDOF_CMP_CD'].fillna('etc', inplace=True)
  tmp =[] # 개수가 얼마나 나왔는지 저장을 위해
  for i in df['HDOF_CMP_CD']:
    tmp.append(list(df['HDOF_CMP_CD']).count(i))
  df['HDOF_CMP_CD'] = tmp



  # 설립일자 널값은 어떻게 채울까, 일단 최빈값으로 채우자
  # 중요한 데이터면 인터넷 검색해서 채우면 된다.
  # df['ESTB_DATE'].fillna(df['ESTB_DATE'].mode(), inplace = True)

  # 설립구분은 널값 없다. 일단 넣는다.
  df['ESTB_GB'] = df['ESTB_GB'].astype('int')

  # 상장일자는 널값인게 매우 많다.
  # 널값이면 0, 아니면 1로 채운다. - 상장했는지 안했는지 여부만 확인하려고 ?
  df['LIST_DATE'].fillna(0, inplace=True)
  df['LIST_DATE'] = df['LIST_DATE'].apply(lambda x: 1 if x!=0 else 0)

  # 주요사업내용
  df['MN_BIZ_CONT'].fillna('etc', inplace=True)

  # 국가명
  # 나중에 gdp나 중소기업 지원정책이나 이런거 파생변수로 넣을 수 있을듯
  df['NATN_NM'] = df['NATN_NM'].fillna('etc')



  # 홈페이지 url
  # 홈페이지 url에 있는지 없는지 여부에 따라 1, 0으로 라벨링
  df['HOMEPAGE_URL'].fillna(0, inplace=True)
  df['HOMEPAGE_URL'] =  df['HOMEPAGE_URL'].apply(lambda x:0 if x==0 else 1)


  # 대표자명
  df['CEO_NM'].fillna('etc', inplace=True)



  # 직원수
  # 널값은 일단 0으로 채운다.
  df['EMP_CNT'].fillna(0, inplace=True)
  


  ########################################################### 설립일자 보류 #############################################
  # 기간을 계산할수있으면 좋을듯 today랑 빼든 종료일자랑 빼든 시작일자랑 빼든
  # 설립일자 널값 어떻게 채워줄까

  ########################################################### 주요사업내용 보류 #########################################
  # 자연어 처리로 활용할 방안 있는지 고려

  ########################################################### 대표자명 보류 #############################################
  # 이것도 대표자명의 건강상태를 통해 파악할만한 것이 있는지 확인




  # active_important, closing_important 전처리 필요
  # 하나의 unique_BIZ_NO 
  # 만약에 하나도 없으면 0, 개수만큼 쓴다. -> 휴폐업구분이 여러개 나오는 것도 있다.
  # for i in active_important['BIZ_NO'].unique():
    


  # for i in closing_important['BIZ_NO'].unique():




  df.reset_index(drop=True, inplace=True)







  return df

In [5]:
df = make_preprocessing(active, closing, active_important, closing_important)

In [6]:
df.head(10)

,BIZ_NO,CMP_NM,BZ_TYP,CMP_SCL,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,휴폐업_여부,(주)여부
0,1138111739,한일가스산업,M,2,1,N,N,0,47712.0,N,31363,19660419.0,1,0,LP Gas 충전 및 판매업,etc,1,홍순철외 1명,43.0,0,1
1,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1
2,1318167565,도영운수,M,2,1,N,N,0,49212.0,N,31363,20011030.0,1,0,시내버스업,etc,1,김명화,149.0,0,1
3,5048128251,중원산업,M,2,1,N,N,0,30391.0,N,31363,19990112.0,1,0,자동차 부품 제조,etc,1,김명구,75.0,0,1
4,1098164260,카타나골프,M,2,1,N,N,0,46800.0,N,31363,20000531.0,1,0,골프용품 전반의 제조 및 수출업,etc,0,김홍득,3.0,0,1
5,2148612093,센트로,M,2,1,N,N,0,55101.0,N,31363,19880810.0,1,0,호텔운영 및 객장임대업,etc,1,이연신,2.0,0,1
6,2068148256,엄앤드이종합건축사사무소,M,2,1,N,N,0,72110.0,N,31363,19940120.0,1,0,"설계,감리",etc,1,이각표외 3명,200.0,0,1
7,1078156099,세풍폴리머,M,2,1,N,N,0,46733.0,N,31363,19960701.0,1,0,플라스틱 합성수지 제조업체,etc,1,김종만,36.0,0,1
8,6218111642,제일산업,M,2,1,N,N,0,etc,N,31363,19931021.0,1,0,마대원단 직조 및 판매업,etc,1,정득부,17.0,0,1
9,5148140835,한국대풍,M,2,1,Y,N,0,30310.0,N,31363,20010621.0,1,0,"자동차부품의 제조,판매,수출입",일본,1,아와츠시게키외 1명,94.0,0,1


In [7]:
# 휴폐업 이력 있는 액티브 데이터 삭제
active_del_list = list(active_important['BIZ_NO'].unique())

del_idx = []
for i in range(len(df)):
  if df['BIZ_NO'][i] in active_del_list:
    del_idx.append(i)

df.drop(del_idx,axis=0, inplace = True)
df = df.reset_index(drop=True)

In [8]:
final_df = pd.merge(df, fs, left_on = 'BIZ_NO', right_on = '사업자번호', how='left')

In [9]:
final_df

,BIZ_NO,CMP_NM,BZ_TYP,CMP_SCL,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,휴폐업_여부,(주)여부,사업자번호,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,자본금,이익잉여금(결손금）,자본총계,매출액,판매비와관리비,영업이익（손실）,법인세비용차감전순손익,법인세비용,당기순이익(손실),기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,영업비율(%),금융비용대매출액비율(%,금융비용대부채비율(%),금융비용대총비용비율(%,부채비율(%),차입금의존도(%),자기자본비율(%),순운전자본비율(%),유동부채비율(%),비유동부채비율(%),부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),미수금,매출원가,무형자산,재고자산
0,1138111739,한일가스산업,M,2,1,N,N,0,47712.0,N,31363,19660419.0,1,0,LP Gas 충전 및 판매업,etc,1,홍순철외 1명,43.0,0,1,1.138112e+09,20211231.0,2758846.0,1577971.0,11235042.0,5705994.0,13993889.0,6172590.0,6343124.0,12515714.0,300000.0,1178175.0,1478175.0,19738142.0,5176975.0,-979131.0,-812712.0,NaN,-812712.0,1.000000e+12,8.419,392.725,2.126800e+01,-4.961000e+00,-4.118000e+00,104.027,-2.099712e+06,1.049610e+02,3.230000e-01,1.000000e+12,0.303,8.467000e+02,24.165,1.056300e+01,-24.395,4.175820e+02,4.291190e+02,6.340900e+01,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,100333.0,15540298.0,25000.0,59170.0
1,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20191231.0,4772148.0,1203628.0,6784222.0,4378120.0,11556370.0,3769840.0,3756536.0,7526376.0,100000.0,3929994.0,4029994.0,18826244.0,1051725.0,1327150.0,1017636.0,205328.0,812308.0,9.200000e+00,34.000,3930.000,1.260000e+01,7.000000e+00,4.300000e+00,94.600,NaN,9.300000e+01,9.000000e-01,2.400000e+00,1.000,1.868000e+02,24.900,3.490000e+01,8.700,9.350000e+01,9.320000e+01,4.000000e+01,1.746000e+00,1.039200e+01,3.128200e+01,4.713400e+01,433886.0,16447369.0,NaN,1792159.0
2,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20201231.0,6096808.0,1229902.0,7819289.0,4307102.0,13916097.0,5397205.0,4348030.0,9745234.0,100000.0,4070863.0,4170863.0,13852434.0,900743.0,-46273.0,170235.0,29366.0,140869.0,2.355000e+00,29.253,4070.863,6.168000e+00,-3.340000e-01,1.017000e+00,98.807,1.000000e+12,1.003340e+02,1.149000e+00,1.842000e+00,1.129,2.336500e+02,40.155,2.997200e+01,5.027,1.294030e+02,1.042480e+02,7.035000e+01,1.088000e+00,6.019000e+00,1.138500e+01,1.269100e+01,445094.0,12997964.0,NaN,2810573.0
3,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20211231.0,5562280.0,1443248.0,8236579.0,4351158.0,13798860.0,5247799.0,4002659.0,9250457.0,100000.0,4448402.0,4548402.0,14889722.0,765533.0,452145.0,391144.0,13605.0,377539.0,3.881000e+00,32.238,4448.402,8.178000e+00,3.037000e+00,2.536000e+00,97.398,-1.352845e+06,9.696300e+01,1.076000e+00,1.687000e+00,1.094,2.033780e+02,40.451,3.296200e+01,2.279,1.153770e+02,8.800100e+01,6.212700e+01,1.074000e+00,5.800000e+00,1.114000e+01,1.271000e+01,429334.0,13672044.0,208.0,2324076.0
4,1318167565,도영운수,M,2,1,N,N,0,49212.0,N,31363,20011030.0,1,0,시내버스업,etc,1,김명화,149.0,0,1,1.318168e+09,20191231.0,1241591.0,990206.0,3341364.0,3195475.0,4582955.0,2349133.0,NaN,2349133.0,200000.0,2033822.0,2233822.0,11341536.0,846751.0,-67323.0,-183472.0,37794.0,-221265.0,-4.640000e+00,44.378,1016.911,6.872000e+00,-5.940000e-01,-1.951000e+00,101.605,1.000000e+12,1.005940e+02,2.000000e-03,1.000000e-02,0.002,1.051620e+02,14.347,4.874200e+01,-24.167,1.051620e+02,1.000000e+12,2.071300e+01,2.381000e+00,1.000000e+12,2.290700e+01,9.378100e+01,165.0,10562108.0,1797.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107962,8018501578,우진푸드중부지점,M,2,2,N,N,0,etc,N,1,20200727.0,1,0,"돈가스, 탕수육, 꿔바로우",etc,1,신남정,22.0,1,1,8.018502e+09,20201231.0,1206945.0,731340.0,6396899.0,6254160.0,7603844.0,3263548.0,4749000.0,8012548.0,400000.0,-808704.0,-40870

In [ ]:
final_df.to_csv('baseline_df.csv', index = False)

# Feature 추가하기


In [57]:
final_df = pd.read_csv('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/baseline_df.csv')

In [58]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107967 entries, 0 to 107966
Data columns (total 67 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   BIZ_NO           107967 non-null  int64  
 1   CMP_NM           107967 non-null  object 
 2   BZ_TYP           107967 non-null  object 
 3   CMP_SCL          107967 non-null  int64  
 4   HDOF_BR_GB       107967 non-null  int64  
 5   FR_IVST_CORP_YN  107967 non-null  object 
 6   VENT_YN          107967 non-null  object 
 7   LIST_CD          107967 non-null  int64  
 8   IND_CD1          107967 non-null  object 
 9   MDSCO_PRTC_YN    107967 non-null  object 
 10  HDOF_CMP_CD      107967 non-null  int64  
 11  ESTB_DATE        106011 non-null  float64
 12  ESTB_GB          107967 non-null  int64  
 13  LIST_DATE        107967 non-null  int64  
 14  MN_BIZ_CONT      107967 non-null  object 
 15  NATN_NM          107967 non-null  object 
 16  HOMEPAGE_URL     107967 non-null  int6

In [59]:
# 모든 재무제표가 null인거 삭제

final_df[final_df['결산년월'].isnull()]

,BIZ_NO,CMP_NM,BZ_TYP,CMP_SCL,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,휴폐업_여부,(주)여부,사업자번호,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,자본금,이익잉여금(결손금）,자본총계,매출액,판매비와관리비,영업이익（손실）,법인세비용차감전순손익,법인세비용,당기순이익(손실),기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,영업비율(%),금융비용대매출액비율(%,금융비용대부채비율(%),금융비용대총비용비율(%,부채비율(%),차입금의존도(%),자기자본비율(%),순운전자본비율(%),유동부채비율(%),비유동부채비율(%),부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),미수금,매출원가,무형자산,재고자산
46445,3038511820,대명상호저축은행(충주지점),A,2,2,N,N,0,etc,N,1,19720201.0,1,0,금융-상호신용,etc,1,이정일외 1명,30.0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64796,6108535846,진주상호저축은행울산지점,A,2,2,N,N,0,etc,N,3,NaN,99,0,상호금융-금융,etc,0,박기권,7.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64810,6098514161,진주상호저축은행,A,2,2,N,N,0,etc,N,3,NaN,1,0,성호금융업-금융업,etc,0,박기권,7.0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81067,5378100097,하늘연,M,2,1,N,N,0,68221.0,N,31363,20150428.0,1,0,부동산 컨설팅업,etc,0,김형래외 1명,2.0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99586,2068607298,글로미트,M,2,1,N,N,0,46313.0,N,31363,20060424.0,1,0,축산물 가공업(식육포장처리업 및 육류가공),etc,0,신임호,3.0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99918,1048610686,덕이드림티와이,V,2,1,N,N,0,64999.0,N,31363,20071017.0,1,0,사채 및 기업어음(ABCP)의 발행 및 상환,etc,0,이희훈,1.0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100194,2158637781,아산상조,M,2,1,N,N,0,96921.0,N,31363,20030301.0,1,0,결혼및장례관련 대행서비스업,etc,1,장재백,3.0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103288,1078821420,엠케이알오케이,O,2,1,N,N,0,64999.0,N,31363,20140613.0,1,0,"국고채, 국내 및 국외 회사가 발행한 사채, 파생결합증권(신용연계 파생",etc,0,김태진,1.0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104007,3488100027,지스타코리아,M,2,1,N,N,0,46799.0,N,31363,20150224.0,1,0,휴대폰 악세서리 및 주변기기 도.소매업,etc,1,김성운,3.0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104347,4378600166,엘리시아제이차,O,2,1,N,N,0,64999.0,N,31363,20150720.0,1,0,"국내 및 국외 회사가 발행한 사채, 주식, 파생결합증권 등 자본시장과 금",etc,0,김윤보,0.0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
# 12개 row 삭제
final_df.drop(list(final_df[final_df['결산년월'].isnull()].index) ,axis=0, inplace=True)
final_df = final_df.reset_index(drop=True)

In [61]:
final_df

,BIZ_NO,CMP_NM,BZ_TYP,CMP_SCL,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,휴폐업_여부,(주)여부,사업자번호,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,자본금,이익잉여금(결손금）,자본총계,매출액,판매비와관리비,영업이익（손실）,법인세비용차감전순손익,법인세비용,당기순이익(손실),기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,영업비율(%),금융비용대매출액비율(%,금융비용대부채비율(%),금융비용대총비용비율(%,부채비율(%),차입금의존도(%),자기자본비율(%),순운전자본비율(%),유동부채비율(%),비유동부채비율(%),부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),미수금,매출원가,무형자산,재고자산
0,1138111739,한일가스산업,M,2,1,N,N,0,47712.0,N,31363,19660419.0,1,0,LP Gas 충전 및 판매업,etc,1,홍순철외 1명,43.0,0,1,1.138112e+09,20211231.0,2758846.0,1577971.0,11235042.0,5705994.0,13993889.0,6172590.0,6343124.0,12515714.0,300000.0,1178175.0,1478175.0,19738142.0,5176975.0,-979131.0,-812712.0,NaN,-812712.0,1.000000e+12,8.419,392.725,2.126800e+01,-4.961000e+00,-4.118000e+00,104.027,-2.099712e+06,1.049610e+02,3.230000e-01,1.000000e+12,0.303,8.467000e+02,24.165,1.056300e+01,-24.395,4.175820e+02,4.291190e+02,6.340900e+01,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,100333.0,15540298.0,25000.0,59170.0
1,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20191231.0,4772148.0,1203628.0,6784222.0,4378120.0,11556370.0,3769840.0,3756536.0,7526376.0,100000.0,3929994.0,4029994.0,18826244.0,1051725.0,1327150.0,1017636.0,205328.0,812308.0,9.200000e+00,34.000,3930.000,1.260000e+01,7.000000e+00,4.300000e+00,94.600,NaN,9.300000e+01,9.000000e-01,2.400000e+00,1.000,1.868000e+02,24.900,3.490000e+01,8.700,9.350000e+01,9.320000e+01,4.000000e+01,1.746000e+00,1.039200e+01,3.128200e+01,4.713400e+01,433886.0,16447369.0,NaN,1792159.0
2,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20201231.0,6096808.0,1229902.0,7819289.0,4307102.0,13916097.0,5397205.0,4348030.0,9745234.0,100000.0,4070863.0,4170863.0,13852434.0,900743.0,-46273.0,170235.0,29366.0,140869.0,2.355000e+00,29.253,4070.863,6.168000e+00,-3.340000e-01,1.017000e+00,98.807,1.000000e+12,1.003340e+02,1.149000e+00,1.842000e+00,1.129,2.336500e+02,40.155,2.997200e+01,5.027,1.294030e+02,1.042480e+02,7.035000e+01,1.088000e+00,6.019000e+00,1.138500e+01,1.269100e+01,445094.0,12997964.0,NaN,2810573.0
3,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20211231.0,5562280.0,1443248.0,8236579.0,4351158.0,13798860.0,5247799.0,4002659.0,9250457.0,100000.0,4448402.0,4548402.0,14889722.0,765533.0,452145.0,391144.0,13605.0,377539.0,3.881000e+00,32.238,4448.402,8.178000e+00,3.037000e+00,2.536000e+00,97.398,-1.352845e+06,9.696300e+01,1.076000e+00,1.687000e+00,1.094,2.033780e+02,40.451,3.296200e+01,2.279,1.153770e+02,8.800100e+01,6.212700e+01,1.074000e+00,5.800000e+00,1.114000e+01,1.271000e+01,429334.0,13672044.0,208.0,2324076.0
4,1318167565,도영운수,M,2,1,N,N,0,49212.0,N,31363,20011030.0,1,0,시내버스업,etc,1,김명화,149.0,0,1,1.318168e+09,20191231.0,1241591.0,990206.0,3341364.0,3195475.0,4582955.0,2349133.0,NaN,2349133.0,200000.0,2033822.0,2233822.0,11341536.0,846751.0,-67323.0,-183472.0,37794.0,-221265.0,-4.640000e+00,44.378,1016.911,6.872000e+00,-5.940000e-01,-1.951000e+00,101.605,1.000000e+12,1.005940e+02,2.000000e-03,1.000000e-02,0.002,1.051620e+02,14.347,4.874200e+01,-24.167,1.051620e+02,1.000000e+12,2.071300e+01,2.381000e+00,1.000000e+12,2.290700e+01,9.378100e+01,165.0,10562108.0,1797.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107950,8018501578,우진푸드중부지점,M,2,2,N,N,0,etc,N,1,20200727.0,1,0,"돈가스, 탕수육, 꿔바로우",etc,1,신남정,22.0,1,1,8.018502e+09,20201231.0,1206945.0,731340.0,6396899.0,6254160.0,7603844.0,3263548.0,4749000.0,8012548.0,400000.0,-808704.0,-40870

### altman k- score

In [62]:
재무제표_col = ['유동자산', '매출채권', '비유동자산', '유형자산', '자산총계', '유동부채',
       '비유동부채', '부  채  총  계', '자본금', '이익잉여금(결손금）', '자본총계', '매출액', '판매비와관리비',
       '영업이익（손실）', '법인세비용차감전순손익', '법인세비용', '당기순이익(손실)', '기업순이익률(%)',
       '유보액/총자산(%)', '유보액/납입자본(%)', '매출액총이익률(%)', '매출액영업이익률(%)', '매출액순이익률(%)',
       '수지비율(%)', '경상수지비율', '영업비율(%)', '금융비용대매출액비율(%', '금융비용대부채비율(%)',
       '금융비용대총비용비율(%', '부채비율(%)', '차입금의존도(%)', '자기자본비율(%)', '순운전자본비율(%)',
       '유동부채비율(%)', '비유동부채비율(%)', '부채총계대 매출액(%)', '총자본회전율(회)', '재고자산회전율(회)',
       '매출채권회전율(회)', '매입채무회전율(회)', '미수금', '매출원가', '무형자산', '재고자산']

fs_df_mean = final_df.groupby('CMP_NM')['유동자산', '매출채권', '비유동자산', '유형자산', '자산총계', '유동부채',
       '비유동부채', '부  채  총  계', '자본금', '이익잉여금(결손금）', '자본총계', '매출액', '판매비와관리비',
       '영업이익（손실）', '법인세비용차감전순손익', '법인세비용', '당기순이익(손실)', '기업순이익률(%)',
       '유보액/총자산(%)', '유보액/납입자본(%)', '매출액총이익률(%)', '매출액영업이익률(%)', '매출액순이익률(%)',
       '수지비율(%)', '경상수지비율', '영업비율(%)', '금융비용대매출액비율(%', '금융비용대부채비율(%)',
       '금융비용대총비용비율(%', '부채비율(%)', '차입금의존도(%)', '자기자본비율(%)', '순운전자본비율(%)',
       '유동부채비율(%)', '비유동부채비율(%)', '부채총계대 매출액(%)', '총자본회전율(회)', '재고자산회전율(회)',
       '매출채권회전율(회)', '매입채무회전율(회)', '미수금', '매출원가', '무형자산', '재고자산'].mean().reset_index()

new_col = []
for i in fs_df_mean.columns:
  new_col.append(i+'_for_fillna')
fs_df_mean.columns = new_col
fs_df_mean

,CMP_NM_for_fillna,유동자산_for_fillna,매출채권_for_fillna,비유동자산_for_fillna,유형자산_for_fillna,자산총계_for_fillna,유동부채_for_fillna,비유동부채_for_fillna,부 채 총 계_for_fillna,자본금_for_fillna,이익잉여금(결손금）_for_fillna,자본총계_for_fillna,매출액_for_fillna,판매비와관리비_for_fillna,영업이익（손실）_for_fillna,법인세비용차감전순손익_for_fillna,법인세비용_for_fillna,당기순이익(손실)_for_fillna,기업순이익률(%)_for_fillna,유보액/총자산(%)_for_fillna,유보액/납입자본(%)_for_fillna,매출액총이익률(%)_for_fillna,매출액영업이익률(%)_for_fillna,매출액순이익률(%)_for_fillna,수지비율(%)_for_fillna,경상수지비율_for_fillna,영업비율(%)_for_fillna,금융비용대매출액비율(%_for_fillna,금융비용대부채비율(%)_for_fillna,금융비용대총비용비율(%_for_fillna,부채비율(%)_for_fillna,차입금의존도(%)_for_fillna,자기자본비율(%)_for_fillna,순운전자본비율(%)_for_fillna,유동부채비율(%)_for_fillna,비유동부채비율(%)_for_fillna,부채총계대 매출액(%)_for_fillna,총자본회전율(회)_for_fillna,재고자산회전율(회)_for_fillna,매출채권회전율(회)_for_fillna,매입채무회전율(회)_for_fillna,미수금_for_fillna,매출원가_for_fillna,무형자산_for_fillna,재고자산_for_fillna
0,111퍼센트,2.263394e+07,6.079906e+06,4.546796e+06,8.305750e+05,2.718074e+07,4.058260e+06,2.347600e+05,4.293019e+06,10000.0,2.932264e+07,2.288772e+07,5.547129e+07,4.949823e+07,1.733625e+07,2.265207e+07,4.185618e+06,1.528909e+07,7.329225e+01,2.500000e+11,2.500002e+11,1.000000e+02,3.208525e+01,3.049750e+01,5.003550e+01,2.499993e+11,2.500000e+11,2.500000e+11,3.650000e-02,2.500000e+11,1.523425e+01,2.500000e+11,8.699575e+01,70.997750,1.405425e+01,1.180500e+00,7.658000e+00,2.418500e+00,1.000000e+12,7.500000e+11,1.000000e+12,1.053023e+06,NaN,1.804817e+05,NaN
1,153로지스틱스,3.902950e+06,1.892625e+06,1.138697e+06,5.827608e+05,5.041647e+06,2.511743e+06,1.168252e+06,3.679995e+06,550000.0,8.116525e+05,1.361652e+06,1.601306e+07,2.858004e+06,2.995725e+05,2.202090e+05,3.773350e+04,1.824752e+05,6.193750e+00,1.673525e+01,3.347062e+02,3.230650e+01,1.774500e+00,1.082500e+00,9.870375e+01,2.499994e+11,9.822550e+01,3.687500e-01,2.041750e+00,3.725000e-01,3.060577e+02,3.935975e+01,2.573325e+01,31.858250,1.905330e+02,1.155250e+02,2.346500e+01,4.215000e+00,1.000000e+12,1.018800e+01,3.001150e+01,1.266500e+03,1.714064e+07,7.238775e+04,NaN
2,2030청년주택,4.490784e+06,NaN,6.036776e+07,6.035747e+07,6.485854e+07,6.355643e+07,1.000000e+06,6.455643e+07,300000.0,2.115000e+03,3.021150e+05,NaN,NaN,NaN,2.304000e+03,2.640000e+02,2.040000e+03,1.000000e+12,3.000000e-03,7.050000e-01,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,0.000000e+00,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,2.136816e+04,9.951200e+01,4.660000e-01,-91.068000,2.103716e+04,3.310000e+02,8.888889e+11,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,3.556321e+06,NaN,NaN,NaN
3,215맨션,1.790073e+05,NaN,1.236013e+06,7.264300e+04,1.415020e+06,9.047937e+05,NaN,9.047937e+05,42969.0,9.488867e+04,5.102267e+05,4.538427e+05,5.473847e+05,-9.354200e+04,-9.774133e+04,NaN,-9.774133e+04,-1.739167e+01,2.172200e+01,9.824050e+02,1.000000e+02,-6.914733e+01,-7.413433e+01,1.745593e+02,3.333333e+11,1.691473e+02,7.730667e+00,2.917333e+00,4.133667e+00,3.333333e+11,6.375500e+01,3.333333e+11,-56.221000,3.333333e+11,1.000000e+12,3.151443e+02,6.520000e-01,1.000000e+12,1.000000e+12,1.000000e+12,1.860567e+04,NaN,1.115660e+06,NaN
4,21세기,6.802736e+06,3.228512e+06,1.732127e+07,1.066784e+07,2.412401e+07,9.346768e+06,3.437912e+06,1.278468e+07,755125.0,8.015192e+06,1.133933e+07,1.566761e+07,2.447212e+06,1.604884e+06,1.703994e+06,2.034888e+05,1.500506e+06,8.715000e+00,4.441875e+01,1.404175e+03,2.606650e+01,9.522500e+00,9.280250e+00,9.005875e+01,4.999984e+11,9.047750e+01,1.908250e+00,2.659250e+00,2.004500e+00,1.125687e+02,4.144300e+01,4.746200e+01,-9.880000,8.207925e+01,3.048950e+01,8.544550e+01,7.280000e-01,1.772500e+01,5.467250e+00,2.685050e+01,7.172750e+04,1.161551e+07,9.890552e+05,986419.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33194,힘멜,1.308890e+05,6.574100e+04,2.805000e+04,8.349000e+03,1.589380e+05,1.276260e+05,NaN,1.276260e+05,30000.0,1.312000e+03,3.131200e+04,3.294350e+05,4.632000e+04,3.210000e+03,1.426000e+03,1.1400

In [63]:
final_df.isna().sum()[23:].head(44)

유동자산              995
매출채권            17627
비유동자산            2013
유형자산             7167
자산총계              985
유동부채             1139
비유동부채           11162
부  채  총  계       1066
자본금              1025
이익잉여금(결손금）       1269
자본총계              993
매출액              5268
판매비와관리비          1371
영업이익（손실）         1092
법인세비용차감전순손익      1276
법인세비용           35163
당기순이익(손실)        1045
기업순이익률(%)          33
유보액/총자산(%)        127
유보액/납입자본(%)       133
매출액총이익률(%)        643
매출액영업이익률(%)       642
매출액순이익률(%)        644
수지비율(%)           157
경상수지비율           5199
영업비율(%)           642
금융비용대매출액비율(%     2102
금융비용대부채비율(%)       94
금융비용대총비용비율(%     1724
부채비율(%)           121
차입금의존도(%)         370
자기자본비율(%)         119
순운전자본비율(%)        119
유동부채비율(%)         132
비유동부채비율(%)        982
부채총계대 매출액(%)      607
총자본회전율(회)          15
재고자산회전율(회)         14
매출채권회전율(회)       2024
매입채무회전율(회)       3306
미수금             27234
매출원가            17384
무형자산            53082
재고자산            28937
dtype: int64

In [64]:
# 재무제표 컬럼 1차적으로 널값 채우기 - 사업자 번호별 분기별 평균으로 채워넣기
final_df = pd.merge(final_df, fs_df_mean, how = 'left', left_on = 'CMP_NM',right_on = 'CMP_NM_for_fillna')
for i in 재무제표_col:
  final_df[i] =  np.where(pd.notnull(final_df[i])==True, final_df[i], final_df[f'{i}_for_fillna'])
  # 재무제표 컬럼 2차적으로 널값 채우기 - 위에서 안채워진 것은 0으로 채우기
  final_df[i].fillna(0, inplace=True)


# _for_fillna 컬럼 삭제

for i in final_df.columns:
  if '_for_fillna' in i:
    final_df.drop(i, axis=1, inplace=True)

final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107955 entries, 0 to 107954
Data columns (total 67 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   BIZ_NO           107955 non-null  int64  
 1   CMP_NM           107955 non-null  object 
 2   BZ_TYP           107955 non-null  object 
 3   CMP_SCL          107955 non-null  int64  
 4   HDOF_BR_GB       107955 non-null  int64  
 5   FR_IVST_CORP_YN  107955 non-null  object 
 6   VENT_YN          107955 non-null  object 
 7   LIST_CD          107955 non-null  int64  
 8   IND_CD1          107955 non-null  object 
 9   MDSCO_PRTC_YN    107955 non-null  object 
 10  HDOF_CMP_CD      107955 non-null  int64  
 11  ESTB_DATE        106001 non-null  float64
 12  ESTB_GB          107955 non-null  int64  
 13  LIST_DATE        107955 non-null  int64  
 14  MN_BIZ_CONT      107955 non-null  object 
 15  NATN_NM          107955 non-null  object 
 16  HOMEPAGE_URL     107955 non-null  int6

In [68]:
# 알트만 K score
def altman_k_score(df):
  df['X1'] = np.log(df['자산총계'])
  df['X2'] = np.log(df['총자본회전율(회)'])
  df['X3'] = df['이익잉여금(결손금）'] / df['자산총계']
  df['X4'] = df['자본총계'] / df['자산총계']

  df['K_score'] = -17.862 + 1.472 * df['X1'] + 3.041 * df['X2'] + 14.839 * df['X3'] + 1.516 * df['X4']

  df.drop(['X1','X2', 'X3', 'X4'], axis=1,inplace=True)

  # 부실 라벨링
  # 0.75 > K: 부실가능성 X
  # -2 < K < 0.75 : 판정 유보
  # K< -2 : 부실가능성 심각

  df['K_부실여부판단'] = df.apply(lambda x: '안전기업' if x['K_score']>0.75 else '판정유보기업' if x['K_score'] > -2 else '부실 가능성 높은 기업', axis=1)

  return df


final_df = altman_k_score(final_df)
final_df

,BIZ_NO,CMP_NM,BZ_TYP,CMP_SCL,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,휴폐업_여부,(주)여부,사업자번호,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,자본금,이익잉여금(결손금）,자본총계,매출액,판매비와관리비,영업이익（손실）,법인세비용차감전순손익,법인세비용,당기순이익(손실),기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,영업비율(%),금융비용대매출액비율(%,금융비용대부채비율(%),금융비용대총비용비율(%,부채비율(%),차입금의존도(%),자기자본비율(%),순운전자본비율(%),유동부채비율(%),비유동부채비율(%),부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),미수금,매출원가,무형자산,재고자산,K_score,K_부실여부판단
0,1138111739,한일가스산업,M,2,1,N,N,0,47712.0,N,31363,19660419.0,1,0,LP Gas 충전 및 판매업,etc,1,홍순철외 1명,43.0,0,1,1.138112e+09,20211231.0,2758846.0,1577971.0,11235042.0,5705994.0,13993889.0,6172590.0,6343124.0,12515714.0,300000.0,1178175.0,1478175.0,19738142.0,5176975.0,-979131.0,-812712.0,0.0,-812712.0,1.000000e+12,8.419,392.725,2.126800e+01,-4.961000e+00,-4.118000e+00,104.027,-2.099712e+06,1.049610e+02,3.230000e-01,1.000000e+12,0.303,8.467000e+02,24.165,1.056300e+01,-24.395,4.175820e+02,4.291190e+02,6.340900e+01,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,100333.0,15540298.0,25000.0,59170.0,91.793878,안전기업
1,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20191231.0,4772148.0,1203628.0,6784222.0,4378120.0,11556370.0,3769840.0,3756536.0,7526376.0,100000.0,3929994.0,4029994.0,18826244.0,1051725.0,1327150.0,1017636.0,205328.0,812308.0,9.200000e+00,34.000,3930.000,1.260000e+01,7.000000e+00,4.300000e+00,94.600,4.999993e+11,9.300000e+01,9.000000e-01,2.400000e+00,1.000,1.868000e+02,24.900,3.490000e+01,8.700,9.350000e+01,9.320000e+01,4.000000e+01,1.746000e+00,1.039200e+01,3.128200e+01,4.713400e+01,433886.0,16447369.0,208.0,1792159.0,13.346587,안전기업
2,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20201231.0,6096808.0,1229902.0,7819289.0,4307102.0,13916097.0,5397205.0,4348030.0,9745234.0,100000.0,4070863.0,4170863.0,13852434.0,900743.0,-46273.0,170235.0,29366.0,140869.0,2.355000e+00,29.253,4070.863,6.168000e+00,-3.340000e-01,1.017000e+00,98.807,1.000000e+12,1.003340e+02,1.149000e+00,1.842000e+00,1.129,2.336500e+02,40.155,2.997200e+01,5.027,1.294030e+02,1.042480e+02,7.035000e+01,1.088000e+00,6.019000e+00,1.138500e+01,1.269100e+01,445094.0,12997964.0,208.0,2810573.0,11.401964,안전기업
3,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20211231.0,5562280.0,1443248.0,8236579.0,4351158.0,13798860.0,5247799.0,4002659.0,9250457.0,100000.0,4448402.0,4548402.0,14889722.0,765533.0,452145.0,391144.0,13605.0,377539.0,3.881000e+00,32.238,4448.402,8.178000e+00,3.037000e+00,2.536000e+00,97.398,-1.352845e+06,9.696300e+01,1.076000e+00,1.687000e+00,1.094,2.033780e+02,40.451,3.296200e+01,2.279,1.153770e+02,8.800100e+01,6.212700e+01,1.074000e+00,5.800000e+00,1.114000e+01,1.271000e+01,429334.0,13672044.0,208.0,2324076.0,11.838342,안전기업
4,1318167565,도영운수,M,2,1,N,N,0,49212.0,N,31363,20011030.0,1,0,시내버스업,etc,1,김명화,149.0,0,1,1.318168e+09,20191231.0,1241591.0,990206.0,3341364.0,3195475.0,4582955.0,2349133.0,1153634.5,2349133.0,200000.0,2033822.0,2233822.0,11341536.0,846751.0,-67323.0,-183472.0,37794.0,-221265.0,-4.640000e+00,44.378,1016.911,6.872000e+00,-5.940000e-01,-1.951000e+00,101.605,1.000000e+12,1.005940e+02,2.000000e-03,1.000000e-02,0.002,1.051620e+02,14.347,4.874200e+01,-24.167,1.051620e+02,1.000000e+12,2.071300e+01,2.381000e+00,1.000000e+12,2.290700e+01,9.378100e+01,165.0,10562108.0,1797.0,0.0,14.677626,안전기업
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107950,8018501578,우진푸드중부지점,M,2,2,N,N,0,etc,N,1,20200727.0,1,0,"돈가스, 탕수육, 꿔바로우",etc,1,신남정,22.0,1,1,8.018502e

In [70]:
final_df['K_score'].isna().sum()
# 0으로 나눈게 있어서 그런가 널값이 나온다.

749

In [71]:
final_df['K_부실여부판단'].isna().sum()
# 널값 나온것들이 모두 부실 가능성 높은 기업으로 분류됐다.

0

In [72]:
final_df['K_부실여부판단'].value_counts()

안전기업            89641
부실 가능성 높은 기업    12854
판정유보기업           5460
Name: K_부실여부판단, dtype: int64

In [76]:
final_df['결산년월'] = final_df['결산년월'].astype('int')
final_df['결산년월'] = final_df['결산년월'].astype('str')


0         20211231.0
1         20191231.0
2         20201231.0
3         20211231.0
4         20191231.0
             ...    
107950    20201231.0
107951    20191231.0
107952    20201231.0
107953    20211231.0
107954    20201231.0
Name: 결산년월, Length: 107955, dtype: float64

In [79]:
# 최소 일자
print(final_df['결산년월'].min())

# 최대 일자 
print(final_df['결산년월'].max())

20180131.0
20220630.0


In [78]:
final_df['결산년월'].max()

20220630.0

In [82]:
!pip install finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [83]:
import FinanceDataReader as fdr

In [84]:
# 원달라 환율
USD_KRX = fdr.DataReader('USD/KRW', start = '2018-01-01') # 미국 달러 / 한국 원(원달러)

In [86]:
USD_KRX = USD_KRX.reset_index()
USD_KRX = USD_KRX[['Date', 'Close']]
USD_KRX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1266 entries, 0 to 1265
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1266 non-null   datetime64[ns]
 1   Close   1265 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 19.9 KB


In [90]:
# 연월별로 평균내기(띄엄띄엄 있다.)
USD_KRX['Date'] = USD_KRX['Date'].astype('str')
USD_KRX['연월'] = USD_KRX.apply(lambda x: x['Date'][:7], axis=1)
USD_KRX

,Date,Close,연월
0,2018-01-01,1066.069946,2018-01
1,2018-01-02,1065.219971,2018-01
2,2018-01-03,1062.540039,2018-01
3,2018-01-04,1063.939941,2018-01
4,2018-01-05,1061.079956,2018-01
...,...,...,...
1261,2022-11-01,1424.670044,2022-11
1262,2022-11-02,1416.599976,2022-11
1263,2022-11-03,1423.099976,2022-11
1264,2022-11-04,1423.650024,2022-11


In [95]:
date = list(USD_KRX.groupby('연월')['Close'].mean().index)
close = list(USD_KRX.groupby('연월')['Close'].mean().values)

환율 = pd.DataFrame()
환율['연월'] = date
환율['환율'] = close
환율

,연월,환율
0,2018-01,1065.765216
1,2018-02,1077.729504
2,2018-03,1071.122281
3,2018-04,1067.560483
4,2018-05,1075.170442
5,2018-06,1093.013811
6,2018-07,1121.465010
7,2018-08,1120.125647
8,2018-09,1120.393012
9,2018-10,1131.073911


In [96]:
final_df

,BIZ_NO,CMP_NM,BZ_TYP,CMP_SCL,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,휴폐업_여부,(주)여부,사업자번호,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,자본금,이익잉여금(결손금）,자본총계,매출액,판매비와관리비,영업이익（손실）,법인세비용차감전순손익,법인세비용,당기순이익(손실),기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,영업비율(%),금융비용대매출액비율(%,금융비용대부채비율(%),금융비용대총비용비율(%,부채비율(%),차입금의존도(%),자기자본비율(%),순운전자본비율(%),유동부채비율(%),비유동부채비율(%),부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),미수금,매출원가,무형자산,재고자산,K_score,K_부실여부판단
0,1138111739,한일가스산업,M,2,1,N,N,0,47712.0,N,31363,19660419.0,1,0,LP Gas 충전 및 판매업,etc,1,홍순철외 1명,43.0,0,1,1.138112e+09,20211231.0,2758846.0,1577971.0,11235042.0,5705994.0,13993889.0,6172590.0,6343124.0,12515714.0,300000.0,1178175.0,1478175.0,19738142.0,5176975.0,-979131.0,-812712.0,0.0,-812712.0,1.000000e+12,8.419,392.725,2.126800e+01,-4.961000e+00,-4.118000e+00,104.027,-2.099712e+06,1.049610e+02,3.230000e-01,1.000000e+12,0.303,8.467000e+02,24.165,1.056300e+01,-24.395,4.175820e+02,4.291190e+02,6.340900e+01,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,100333.0,15540298.0,25000.0,59170.0,91.793878,안전기업
1,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20191231.0,4772148.0,1203628.0,6784222.0,4378120.0,11556370.0,3769840.0,3756536.0,7526376.0,100000.0,3929994.0,4029994.0,18826244.0,1051725.0,1327150.0,1017636.0,205328.0,812308.0,9.200000e+00,34.000,3930.000,1.260000e+01,7.000000e+00,4.300000e+00,94.600,4.999993e+11,9.300000e+01,9.000000e-01,2.400000e+00,1.000,1.868000e+02,24.900,3.490000e+01,8.700,9.350000e+01,9.320000e+01,4.000000e+01,1.746000e+00,1.039200e+01,3.128200e+01,4.713400e+01,433886.0,16447369.0,208.0,1792159.0,13.346587,안전기업
2,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20201231.0,6096808.0,1229902.0,7819289.0,4307102.0,13916097.0,5397205.0,4348030.0,9745234.0,100000.0,4070863.0,4170863.0,13852434.0,900743.0,-46273.0,170235.0,29366.0,140869.0,2.355000e+00,29.253,4070.863,6.168000e+00,-3.340000e-01,1.017000e+00,98.807,1.000000e+12,1.003340e+02,1.149000e+00,1.842000e+00,1.129,2.336500e+02,40.155,2.997200e+01,5.027,1.294030e+02,1.042480e+02,7.035000e+01,1.088000e+00,6.019000e+00,1.138500e+01,1.269100e+01,445094.0,12997964.0,208.0,2810573.0,11.401964,안전기업
3,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20211231.0,5562280.0,1443248.0,8236579.0,4351158.0,13798860.0,5247799.0,4002659.0,9250457.0,100000.0,4448402.0,4548402.0,14889722.0,765533.0,452145.0,391144.0,13605.0,377539.0,3.881000e+00,32.238,4448.402,8.178000e+00,3.037000e+00,2.536000e+00,97.398,-1.352845e+06,9.696300e+01,1.076000e+00,1.687000e+00,1.094,2.033780e+02,40.451,3.296200e+01,2.279,1.153770e+02,8.800100e+01,6.212700e+01,1.074000e+00,5.800000e+00,1.114000e+01,1.271000e+01,429334.0,13672044.0,208.0,2324076.0,11.838342,안전기업
4,1318167565,도영운수,M,2,1,N,N,0,49212.0,N,31363,20011030.0,1,0,시내버스업,etc,1,김명화,149.0,0,1,1.318168e+09,20191231.0,1241591.0,990206.0,3341364.0,3195475.0,4582955.0,2349133.0,1153634.5,2349133.0,200000.0,2033822.0,2233822.0,11341536.0,846751.0,-67323.0,-183472.0,37794.0,-221265.0,-4.640000e+00,44.378,1016.911,6.872000e+00,-5.940000e-01,-1.951000e+00,101.605,1.000000e+12,1.005940e+02,2.000000e-03,1.000000e-02,0.002,1.051620e+02,14.347,4.874200e+01,-24.167,1.051620e+02,1.000000e+12,2.071300e+01,2.381000e+00,1.000000e+12,2.290700e+01,9.378100e+01,165.0,10562108.0,1797.0,0.0,14.677626,안전기업
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107950,8018501578,우진푸드중부지점,M,2,2,N,N,0,etc,N,1,20200727.0,1,0,"돈가스, 탕수육, 꿔바로우",etc,1,신남정,22.0,1,1,8.018502e

In [98]:
final_df['연월'] = final_df['결산년월'].astype('str')
final_df['연월'] = final_df.apply(lambda x: x['연월'][:4]+'-'+x['연월'][4:6], axis=1)
final_df

,BIZ_NO,CMP_NM,BZ_TYP,CMP_SCL,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,휴폐업_여부,(주)여부,사업자번호,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,자본금,이익잉여금(결손금）,자본총계,매출액,판매비와관리비,영업이익（손실）,법인세비용차감전순손익,법인세비용,당기순이익(손실),기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,영업비율(%),금융비용대매출액비율(%,금융비용대부채비율(%),금융비용대총비용비율(%,부채비율(%),차입금의존도(%),자기자본비율(%),순운전자본비율(%),유동부채비율(%),비유동부채비율(%),부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),미수금,매출원가,무형자산,재고자산,K_score,K_부실여부판단,연월
0,1138111739,한일가스산업,M,2,1,N,N,0,47712.0,N,31363,19660419.0,1,0,LP Gas 충전 및 판매업,etc,1,홍순철외 1명,43.0,0,1,1.138112e+09,20211231.0,2758846.0,1577971.0,11235042.0,5705994.0,13993889.0,6172590.0,6343124.0,12515714.0,300000.0,1178175.0,1478175.0,19738142.0,5176975.0,-979131.0,-812712.0,0.0,-812712.0,1.000000e+12,8.419,392.725,2.126800e+01,-4.961000e+00,-4.118000e+00,104.027,-2.099712e+06,1.049610e+02,3.230000e-01,1.000000e+12,0.303,8.467000e+02,24.165,1.056300e+01,-24.395,4.175820e+02,4.291190e+02,6.340900e+01,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,100333.0,15540298.0,25000.0,59170.0,91.793878,안전기업,2021-12
1,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20191231.0,4772148.0,1203628.0,6784222.0,4378120.0,11556370.0,3769840.0,3756536.0,7526376.0,100000.0,3929994.0,4029994.0,18826244.0,1051725.0,1327150.0,1017636.0,205328.0,812308.0,9.200000e+00,34.000,3930.000,1.260000e+01,7.000000e+00,4.300000e+00,94.600,4.999993e+11,9.300000e+01,9.000000e-01,2.400000e+00,1.000,1.868000e+02,24.900,3.490000e+01,8.700,9.350000e+01,9.320000e+01,4.000000e+01,1.746000e+00,1.039200e+01,3.128200e+01,4.713400e+01,433886.0,16447369.0,208.0,1792159.0,13.346587,안전기업,2019-12
2,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20201231.0,6096808.0,1229902.0,7819289.0,4307102.0,13916097.0,5397205.0,4348030.0,9745234.0,100000.0,4070863.0,4170863.0,13852434.0,900743.0,-46273.0,170235.0,29366.0,140869.0,2.355000e+00,29.253,4070.863,6.168000e+00,-3.340000e-01,1.017000e+00,98.807,1.000000e+12,1.003340e+02,1.149000e+00,1.842000e+00,1.129,2.336500e+02,40.155,2.997200e+01,5.027,1.294030e+02,1.042480e+02,7.035000e+01,1.088000e+00,6.019000e+00,1.138500e+01,1.269100e+01,445094.0,12997964.0,208.0,2810573.0,11.401964,안전기업,2020-12
3,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20211231.0,5562280.0,1443248.0,8236579.0,4351158.0,13798860.0,5247799.0,4002659.0,9250457.0,100000.0,4448402.0,4548402.0,14889722.0,765533.0,452145.0,391144.0,13605.0,377539.0,3.881000e+00,32.238,4448.402,8.178000e+00,3.037000e+00,2.536000e+00,97.398,-1.352845e+06,9.696300e+01,1.076000e+00,1.687000e+00,1.094,2.033780e+02,40.451,3.296200e+01,2.279,1.153770e+02,8.800100e+01,6.212700e+01,1.074000e+00,5.800000e+00,1.114000e+01,1.271000e+01,429334.0,13672044.0,208.0,2324076.0,11.838342,안전기업,2021-12
4,1318167565,도영운수,M,2,1,N,N,0,49212.0,N,31363,20011030.0,1,0,시내버스업,etc,1,김명화,149.0,0,1,1.318168e+09,20191231.0,1241591.0,990206.0,3341364.0,3195475.0,4582955.0,2349133.0,1153634.5,2349133.0,200000.0,2033822.0,2233822.0,11341536.0,846751.0,-67323.0,-183472.0,37794.0,-221265.0,-4.640000e+00,44.378,1016.911,6.872000e+00,-5.940000e-01,-1.951000e+00,101.605,1.000000e+12,1.005940e+02,2.000000e-03,1.000000e-02,0.002,1.051620e+02,14.347,4.874200e+01,-24.167,1.051620e+02,1.000000e+12,2.071300e+01,2.381000e+00,1.000000e+12,2.290700e+01,9.378100e+01,165.0,10562108.0,1797.0,0.0,14.677626,안전기업,2019-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107950,8018501578,우진푸드중부지점,M,2,2,N,N,0,etc,N,1,20200727.0,1,

In [100]:
final_df = pd.merge(final_df, 환율, how = 'left', on = '연월')
final_df.drop('연월',axis=1, inplace=True)
final_df

,BIZ_NO,CMP_NM,BZ_TYP,CMP_SCL,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,휴폐업_여부,(주)여부,사업자번호,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,자본금,이익잉여금(결손금）,자본총계,매출액,판매비와관리비,영업이익（손실）,법인세비용차감전순손익,법인세비용,당기순이익(손실),기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,영업비율(%),금융비용대매출액비율(%,금융비용대부채비율(%),금융비용대총비용비율(%,부채비율(%),차입금의존도(%),자기자본비율(%),순운전자본비율(%),유동부채비율(%),비유동부채비율(%),부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),미수금,매출원가,무형자산,재고자산,K_score,K_부실여부판단,환율_x,환율_y
0,1138111739,한일가스산업,M,2,1,N,N,0,47712.0,N,31363,19660419.0,1,0,LP Gas 충전 및 판매업,etc,1,홍순철외 1명,43.0,0,1,1.138112e+09,20211231.0,2758846.0,1577971.0,11235042.0,5705994.0,13993889.0,6172590.0,6343124.0,12515714.0,300000.0,1178175.0,1478175.0,19738142.0,5176975.0,-979131.0,-812712.0,0.0,-812712.0,1.000000e+12,8.419,392.725,2.126800e+01,-4.961000e+00,-4.118000e+00,104.027,-2.099712e+06,1.049610e+02,3.230000e-01,1.000000e+12,0.303,8.467000e+02,24.165,1.056300e+01,-24.395,4.175820e+02,4.291190e+02,6.340900e+01,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,100333.0,15540298.0,25000.0,59170.0,91.793878,안전기업,1183.480442,1183.480442
1,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20191231.0,4772148.0,1203628.0,6784222.0,4378120.0,11556370.0,3769840.0,3756536.0,7526376.0,100000.0,3929994.0,4029994.0,18826244.0,1051725.0,1327150.0,1017636.0,205328.0,812308.0,9.200000e+00,34.000,3930.000,1.260000e+01,7.000000e+00,4.300000e+00,94.600,4.999993e+11,9.300000e+01,9.000000e-01,2.400000e+00,1.000,1.868000e+02,24.900,3.490000e+01,8.700,9.350000e+01,9.320000e+01,4.000000e+01,1.746000e+00,1.039200e+01,3.128200e+01,4.713400e+01,433886.0,16447369.0,208.0,1792159.0,13.346587,안전기업,1172.771817,1172.771817
2,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20201231.0,6096808.0,1229902.0,7819289.0,4307102.0,13916097.0,5397205.0,4348030.0,9745234.0,100000.0,4070863.0,4170863.0,13852434.0,900743.0,-46273.0,170235.0,29366.0,140869.0,2.355000e+00,29.253,4070.863,6.168000e+00,-3.340000e-01,1.017000e+00,98.807,1.000000e+12,1.003340e+02,1.149000e+00,1.842000e+00,1.129,2.336500e+02,40.155,2.997200e+01,5.027,1.294030e+02,1.042480e+02,7.035000e+01,1.088000e+00,6.019000e+00,1.138500e+01,1.269100e+01,445094.0,12997964.0,208.0,2810573.0,11.401964,안전기업,1094.472178,1094.472178
3,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20211231.0,5562280.0,1443248.0,8236579.0,4351158.0,13798860.0,5247799.0,4002659.0,9250457.0,100000.0,4448402.0,4548402.0,14889722.0,765533.0,452145.0,391144.0,13605.0,377539.0,3.881000e+00,32.238,4448.402,8.178000e+00,3.037000e+00,2.536000e+00,97.398,-1.352845e+06,9.696300e+01,1.076000e+00,1.687000e+00,1.094,2.033780e+02,40.451,3.296200e+01,2.279,1.153770e+02,8.800100e+01,6.212700e+01,1.074000e+00,5.800000e+00,1.114000e+01,1.271000e+01,429334.0,13672044.0,208.0,2324076.0,11.838342,안전기업,1183.480442,1183.480442
4,1318167565,도영운수,M,2,1,N,N,0,49212.0,N,31363,20011030.0,1,0,시내버스업,etc,1,김명화,149.0,0,1,1.318168e+09,20191231.0,1241591.0,990206.0,3341364.0,3195475.0,4582955.0,2349133.0,1153634.5,2349133.0,200000.0,2033822.0,2233822.0,11341536.0,846751.0,-67323.0,-183472.0,37794.0,-221265.0,-4.640000e+00,44.378,1016.911,6.872000e+00,-5.940000e-01,-1.951000e+00,101.605,1.000000e+12,1.005940e+02,2.000000e-03,1.000000e-02,0.002,1.051620e+02,14.347,4.874200e+01,-24.167,1.051620e+02,1.000000e+12,2.071300e+01,2.381000e+00,1.000000e+12,2.290700e+01,9.378100e+01,165.0,10562108.0,1797.0,0.0,14.677626,안전기업,1172.771817,1172.771817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
print(len(del_idx)) # 외감 데이터에서는 362개만 일대일 매칭.
print(len(df)) # 362개를 지운 최종 데이터프레임 길이.
print(len(df['BIZ_NO'].unique()))

362
35359
35359


In [ ]:
df['ESTB_DATE'].value_counts()

20000701.0    45
20000101.0    42
19990101.0    42
20040701.0    37
19991001.0    36
              ..
19860128.0     1
19991010.0     1
20010519.0     1
20040205.0     1
20200629.0     1
Name: ESTB_DATE, Length: 9871, dtype: int64

In [ ]:
# 최종 df와 재무제표 데이터 일대일 매칭되는지 확인 (merge 이전)
no_match_biz_no = []
for i in list(df['BIZ_NO']):
  if i not in list(fs['사업자번호'].unique()):
    no_match_biz_no.append(i)

print(len(no_match_biz_no))

  

12


12개만 재무제표 데이터가 없다.

In [ ]:
final_df = pd.merge(df, fs, left_on = 'BIZ_NO', right_on = '사업자번호', how='left')

In [ ]:
print(len(final_df))
print(len(final_df['BIZ_NO'].unique()))
# 각 사업자번호별로 여러개의 재무제표를 갖는 데이터가 있다.

107967
35359


In [ ]:
final_df.isna().sum()

BIZ_NO            0
CMP_NM            0
BZ_TYP            0
CMP_SCL           0
HDOF_BR_GB        0
              ...  
매입채무회전율(회)     3318
미수금           27246
매출원가          17396
무형자산          53094
재고자산          28949
Length: 68, dtype: int64

In [ ]:
df['휴폐업_여부'].value_counts()

0    28620
1     6739
Name: 휴폐업_여부, dtype: int64

In [ ]:
fs.isna().sum()

사업자번호               0
결산년월                0
유동자산              999
매출채권            18010
비유동자산            2025
유형자산             7244
자산총계              989
유동부채             1144
비유동부채           11260
부  채  총  계       1070
자본금              1029
이익잉여금(결손금）       1273
자본총계              997
매출액              5408
판매비와관리비          1375
영업이익（손실）         1096
법인세비용차감전순손익      1280
법인세비용           35684
당기순이익(손실)        1049
기업순이익률(%)          33
유보액/총자산(%)        127
유보액/납입자본(%)       133
매출액총이익률(%)        660
매출액영업이익률(%)       659
매출액순이익률(%)        661
수지비율(%)           159
경상수지비율           5277
영업비율(%)           659
금융비용대매출액비율(%     2135
금융비용대부채비율(%)       94
금융비용대총비용비율(%     1743
부채비율(%)           121
차입금의존도(%)         376
자기자본비율(%)         119
순운전자본비율(%)        119
유동부채비율(%)         132
비유동부채비율(%)        993
부채총계대 매출액(%)      624
총자본회전율(회)          15
재고자산회전율(회)         14
매출채권회전율(회)       2068
매입채무회전율(회)       3376
미수금             27548
매출원가            17769
무형자산            53849
재고자산      

In [ ]:
# 설립일자 널값 채우기
def fill_ESTB_DATE(df):
  

In [ ]:
df.columns

Index(['BIZ_NO', 'CMP_NM', 'BZ_TYP', 'CMP_SCL', 'HDOF_BR_GB',
       'FR_IVST_CORP_YN', 'VENT_YN', 'LIST_CD', 'IND_CD1', 'MDSCO_PRTC_YN',
       'HDOF_CMP_CD', 'ESTB_DATE', 'ESTB_GB', 'LIST_DATE', 'MN_BIZ_CONT',
       'NATN_NM', 'HOMEPAGE_URL', 'CEO_NM', 'EMP_CNT', '휴폐업_여부', 'tmp',
       '(주)여부'],
      dtype='object')

# 쓸만한 변수
<pre>
1) 기업 존속기간 
- 설립일자 널값인 것은 재무제표 맨처음 수집된 것으로 임시로 채운다.(기업이 안정된 날짜로 판단하고 현재날짜에서 빼서 존속기간 구한다.)
- closing은 closing_important에서 휴폐업 일자를 가져온후 존속기간 구한다.

2) 

In [ ]:
active_important['CLSBZ_GB'].value_counts()

6    1264
3    1239
2    1168
5    1155
4       1
Name: CLSBZ_GB, dtype: int64

In [ ]:
fs

,사업자번호,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,자본금,이익잉여금(결손금）,자본총계,매출액,판매비와관리비,영업이익（손실）,법인세비용차감전순손익,법인세비용,당기순이익(손실),기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,영업비율(%),금융비용대매출액비율(%,금융비용대부채비율(%),금융비용대총비용비율(%,부채비율(%),차입금의존도(%),자기자본비율(%),순운전자본비율(%),유동부채비율(%),비유동부채비율(%),부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),미수금,매출원가,무형자산,재고자산
0,1018100340,20181231,2667731.0,127320.0,32346444.0,1179096.0,35014174.0,422872.0,16176067.0,16598939.0,721100.0,3859072.0,18415235.0,6629483.0,6210864.0,418619.0,3144847.0,635665.0,2509182.0,6.161,4.877,236.820,1.000000e+02,6.315000e+00,3.784900e+01,66.562,6.107600e+01,9.368500e+01,1.000000e+12,0.000,1.000000e+12,90.137,1.000000e+12,5.259400e+01,6.411,2.296000e+00,8.784100e+01,2.503810e+02,1.630000e-01,1.000000e+12,4.406200e+01,1.000000e+12,NaN,NaN,NaN,NaN
1,1018100340,20191231,2168837.0,94085.0,34151633.0,1209369.0,36320470.0,10553878.0,5240445.0,15794323.0,721100.0,3708092.0,20526147.0,6541583.0,5698511.0,843072.0,3446082.0,712661.0,2733421.0,7.600,4.300,215.900,1.000000e+02,1.290000e+01,4.180000e+01,62.600,NaN,8.710000e+01,NaN,0.000,NaN,76.900,0.000000e+00,5.650000e+01,-23.100,5.140000e+01,2.550000e+01,2.414000e+02,1.840000e-01,1.000000e+12,5.909200e+01,NaN,NaN,NaN,NaN,NaN
2,1018100340,20201231,2272260.0,78612.0,38329680.0,1091153.0,40601940.0,863144.0,15496117.0,16359261.0,721100.0,4230449.0,24242679.0,5933317.0,5865171.0,68146.0,4180173.0,773416.0,3406757.0,8.858,5.121,288.321,1.000000e+02,1.149000e+00,5.741700e+01,58.602,-1.336705e+06,9.885200e+01,1.000000e+12,0.000,1.000000e+12,67.481,1.000000e+12,5.970800e+01,3.471,3.560000e+00,6.392100e+01,2.757190e+02,1.540000e-01,1.000000e+12,6.871400e+01,1.000000e+12,NaN,NaN,NaN,NaN
3,1018100340,20211231,1175030.0,273522.0,37135811.0,993025.0,38310842.0,9494272.0,4901383.0,14395654.0,721100.0,5169522.0,23915187.0,6158586.0,5771585.0,387000.0,4731471.0,907999.0,3823473.0,9.690,7.878,418.549,1.000000e+02,6.284000e+00,6.208400e+01,55.228,-6.344112e+05,9.371600e+01,1.000000e+12,0.000,1.000000e+12,60.195,1.000000e+12,6.242400e+01,-21.715,3.970000e+01,2.049500e+01,2.337490e+02,1.560000e-01,1.000000e+12,3.497900e+01,1.000000e+12,3243.0,NaN,NaN,NaN
4,1018100772,20181231,422146.0,17011.0,20370096.0,20352846.0,20792242.0,2684934.0,11404637.0,14089571.0,52000.0,-7410129.0,6702671.0,1127092.0,446655.0,155690.0,20159.0,NaN,20159.0,1.231,-32.209,-12878.763,5.344200e+01,1.381300e+01,1.789000e+00,98.361,9.849600e+01,8.618600e+01,2.059900e+01,1.682,1.918600e+01,210.208,3.834900e+01,3.223600e+01,-10.883,4.005800e+01,1.701510e+02,1.250082e+03,5.500000e-02,1.000000e+12,6.971100e+01,1.000000e+12,386880.0,524746.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109137,8998800427,20211231,9192992.0,2918329.0,9735232.0,9409374.0,18928223.0,11017981.0,6619380.0,17637362.0,514500.0,-1183183.0,1290862.0,25163483.0,3339379.0,-1546667.0,-2254908.0,NaN,-2254908.0,-11.505,4.102,150.896,7.124000e+00,-6.147000e+00,-8.961000e+00,108.783,-5.390522e+06,1.061470e+02,1.172000e+00,2.001,1.056000e+00,1366.324,5.320700e+01,6.820000e+00,-9.642,8.535370e+02,5.127880e+02,7.009100e+01,1.477000e+00,6.785000e+00,7.950000e+00,9.792000e+00,331735.0,23370771.0,75451.0,3680509.0
109138,8998800785,20181231,3608435.0,NaN,271541001.0,271541001.0,275149436.0,164335821.0,76480438.0,240816259.0,37780000.0,-3315168.0,34333177.0,NaN,2612134.0,-2612134.0,-2592341.0,NaN,-2592341.0,-1.690,-1.205,-8.775,1.000000e+12,1.000000e+12,1.000000e+12,12726.477,1.272648e+04,1.000000e+12,1.000000e+12,0.000,1.000000e+12,701.410,8.333700e+01,1.000000e+12,-58.415,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,NaN,NaN,NaN,NaN
109139,8998800785,20191231,475283.0,NaN,309796969.0,309796969.0,310272251.0,3722434.0,274776917.0,278499351.0,37780000.0,-5875445.0,31772900.0,NaN,2561631.0,-25616

In [ ]:
cnt_tmp = []
for i in active_important['BIZ_NO'].unique():
  if i in active['BIZ_NO']:
    cnt_tmp.append(i)
print(len(cnt_tmp))

0


In [ ]:
active

,BIZ_NO,CMP_PFIX_NM,CMP_NM,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,PB_ORG_TYP,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,휴폐업_여부
0,1138111739,NaN,한일가스산업,(주),"HANIL GAS IND. CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,47712.0,NaN,NaN,0,N,NaN,19660419.0,1,NaN,LP Gas 충전 및 판매업,NaN,www.hanilgas.com,홍순철외 1명,43.0,0
1,2078132193,(주),엠피아이,NaN,"EMPIALEE CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,15190.0,NaN,NaN,0,N,NaN,19910806.0,1,NaN,가죽 및 대용가죽제품 제조업,NaN,www.mastercnd.co.kr,이스테판상수외 1명,35.0,0
2,1318167565,NaN,도영운수,(주),"DOYOUNG TRANSPORTATION CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,49212.0,NaN,NaN,0,N,NaN,20011030.0,1,NaN,시내버스업,NaN,transport@naver.com,김명화,149.0,0
3,5048128251,(주),중원산업,NaN,"JOONGWON INDUSTRIAL CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,30391.0,NaN,NaN,0,N,NaN,19990112.0,1,NaN,자동차 부품 제조,NaN,www.koreajw.com,김명구,75.0,0
4,1098164260,(주),카타나골프,NaN,"CATANA GOLF CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,46800.0,NaN,NaN,0,N,NaN,20000531.0,1,NaN,골프용품 전반의 제조 및 수출업,NaN,NaN,김홍득,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28977,8048101453,NaN,제이앤인베스트먼트,NaN,J&INVESTMENT,M,2,2,1,1,N,N,NaN,NaN,NaN,NaN,NaN,0,N,NaN,20191206.0,1,NaN,경영컨설팅업,NaN,NaN,정경태,0.0,0
28978,8028701430,NaN,디앤엠개발,(주),"DNM Development Co., Ltd.",M,2,2,1,1,N,N,NaN,10.0,68122.0,NaN,NaN,0,N,NaN,20200402.0,1,NaN,비주거용 건물 임대업,NaN,NaN,송성현,1.0,0
28979,4268501421,NaN,마스비 건축연구소,NaN,NaN,M,2,2,1,2,N,N,NaN,10.0,72100.0,NaN,NaN,0,N,1220726.0,19900108.0,1,NaN,"BIM용역, 엔지니어링컨설팅",NaN,NaN,김태남,81.0,0
28980,6518502462,(주),건설레미콘의정부공장,NaN,"Construction Remicon Uijeongbu Factory Co., Ltd.",M,2,2,1,2,N,N,NaN,10.0,23322.0,NaN,NaN,0,N,1583116.0,20220615.0,1,NaN,레미콘,NaN,NaN,이미성,26.0,0


In [ ]:
closing_important['CLSBZ_GB'].value_counts()
# 개수가 뭔가 잘못됐다.

3    6838
5       1
6       1
Name: CLSBZ_GB, dtype: int64

- 재무제표가 없는 기업도 있다. 이건 어떻게 고려할건가.
- 외부감사 액티브 데이터는 모두 재무제표 데이터가 거의 있는 것을 확인했다.

In [ ]:
# 각 데이터에서 재무제표 데이터가 얼마나 없는지 확인

fs_BIZ = list(fs['사업자번호'].unique())
# 먼저 active
active_no_fs = []
for i in active['BIZ_NO'].unique():
  if i not in fs_BIZ:
    active_no_fs.append(i)

closing_no_fs = []
for j in closing['BIZ_NO'].unique():
  if j not in fs_BIZ:
    closing_no_fs.append(j)

print(len(active_no_fs))
print(len(closing_no_fs))


# 그럼

4
8


# 짜잘한 EDA

In [ ]:
active = make_preprocessing(active)
closing = make_preprocessing(closing)

In [ ]:
# 국가명
# 나중에 국가별로 경제상황이나 중소기업 지원정책이나 끌어올 수 있으면 좋을듯
# 일단 널값 etc로 채워서 넣는다.
print(active['NATN_NM'].value_counts())
print(closing['NATN_NM'].value_counts())

일본        379
미국        276
싱가포르      146
네덜란드      104
홍콩        103
         ... 
도미니카연방      1
방글라데시       1
폴란드         1
우즈베키스탄      1
라트비아        1
Name: NATN_NM, Length: 63, dtype: int64
일본        41
싱가포르      15
미국        15
홍콩        12
중국         8
독일         7
네덜란드       7
케이만군도      6
버진아일랜드     4
영국         4
스위스        4
몰타         3
대만         2
룩셈부르크      2
캐나다        2
브루나이       1
필리핀        1
기타(구주)     1
벨기에        1
말레이시아      1
오스트리아      1
인도네시아      1
덴마크        1
노르웨이       1
이스라엘       1
호주         1
인도         1
Name: NATN_NM, dtype: int64


In [ ]:
print(len(closing_important))
print(len(active_important))

print()

print(len(closing_important['BIZ_NO'].unique()))
print(len(active_important['BIZ_NO'].unique()))

print()

print(len(closing))
print(len(active))

6840
4827

6746
2083

6739
28982


In [ ]:
# closing에서는 모든 기업들이 closing_important에 있따.
tmp = []
for i in closing['BIZ_NO'].unique():
  if i not in list(closing_important['BIZ_NO'].unique()):
    tmp.append(i)
print(len(tmp))

# active에서는 많은 기업들이 active_important에 없다. 
tmp = []
for i in active['BIZ_NO'].unique():
  if i not in list(active_important['BIZ_NO'].unique()):
    tmp.append(i)
print(len(tmp))

0
28620


# 컬럼 전처리 한 논리
- 아래꺼 보면 이해할 수 있다.

In [ ]:
active = make_preprocessing(active)
active['(주)여부'].value_counts()

1    27374
0     1608
Name: (주)여부, dtype: int64

In [ ]:
closing = make_preprocessing(closing)
closing['(주)여부'].value_counts()

1    5987
0     752
Name: (주)여부, dtype: int64

In [ ]:
active['BZ_TYP'].value_counts()

M    28617
O      267
A       49
V       37
S        8
F        2
I        1
B        1
Name: BZ_TYP, dtype: int64

In [ ]:
closing['BZ_TYP'].value_counts()

M    6698
O      38
V       3
Name: BZ_TYP, dtype: int64

In [ ]:
# 기업규모를 확인해보면 모두 2번으로 일반기업이 나온다.
# 모델링에 큰 영향을 안주는 코드이므로 drop한다.
print(active['CMP_SCL'].value_counts())
print(closing['CMP_SCL'].value_counts())

2    28982
Name: CMP_SCL, dtype: int64
2    6739
Name: CMP_SCL, dtype: int64


In [ ]:
# 공기업구분여부도 모델에 큰영향을 줄 것 같지 않으므로 drop한다.
print(active['PBCO_GB'].value_counts())
print(closing['PBCO_GB'].value_counts())

2    28975
1        7
Name: PBCO_GB, dtype: int64
2    6739
Name: PBCO_GB, dtype: int64


In [ ]:
# 개인법인구분에서 모드 법인으로 나오므로 이것도 컬럼 삭제한다. - 분류모델에 유의미한 영향을 주지 못한다.
print(active['PSN_CORP_GB'].value_counts())
print(closing['PSN_CORP_GB'].value_counts())

1    28982
Name: PSN_CORP_GB, dtype: int64
1    6739
Name: PSN_CORP_GB, dtype: int64


In [ ]:
# 본점지점구분은 그래도 좀 나오니깐 일단 컬럼으로 넣어보고 성능에 유의미한 영향을 주지않으면 빼자
print(active['HDOF_BR_GB'].value_counts())
print(closing['HDOF_BR_GB'].value_counts())

1    26070
2     2912
Name: HDOF_BR_GB, dtype: int64
1    5274
2    1465
Name: HDOF_BR_GB, dtype: int64


In [ ]:
# 국외투자법인여부는 나름 유의미하게 분포되어 있는 것 같다. 효과는 미비할 것 같지만 그나마 쓸만할 것 같아서 빼지않는다.
print(active['FR_IVST_CORP_YN'].value_counts())
print(closing['FR_IVST_CORP_YN'].value_counts())

In [ ]:
# 벤처기업여부 또한 나름 유의미하게 분포
print(active['VENT_YN'].value_counts())
print(closing['VENT_YN'].value_counts())

In [ ]:
# 상장코드는 있는게 매우 드믈다.
# 그래도 혹시 모르니 있으면 1 없으면 0으로 채워넣자.
# 추가적으로 주가 데이터를 활용할 수 있긴한데 데이터가 너무 없어서..애매하다.

print(len(active))
print(active['LIST_CD'].isna().sum())

print(len(closing))
print(closing['LIST_CD'].isna().sum())

28982
28893
6739
6693


In [ ]:
# 산업코드 
# 10개정도 빼고 다 '10'번으로 돼있다. 그냥 drop
print(active['IND_CD_ORDR'].value_counts())
print(closing['IND_CD_ORDR'].value_counts())

10.0    28972
Name: IND_CD_ORDR, dtype: int64
10.0    6734
Name: IND_CD_ORDR, dtype: int64


In [ ]:
# 산업코드1은 unique 라벨값으로 액티브는 1360개, 휴폐업은 1370개가 나온다.
# 이건 아마 1개만 갖는 업종수가 많아서 일 수 있다. 
print(active['IND_CD1'].value_counts())
print(closing['IND_CD1'].value_counts())
print()
print('=================널값확인====================')
print()
print(active['IND_CD1'].isna().sum())
print(closing['IND_CD1'].isna().sum())


print()
print()
print('================10개 이하로 갖고있는 업종코드 확인===============')
tmp = []
for i in range(len(active['IND_CD1'].value_counts())):
  if active['IND_CD1'].value_counts().values[i]<=10:
    tmp.append(active['IND_CD1'].value_counts().index)
print(f' active데이터에서 10개이하로 갖는 업종코드 : {len(tmp)}')

tmp = []
for i in range(len(closing['IND_CD1'].value_counts())):
  if closing['IND_CD1'].value_counts().values[i]<=10:
    tmp.append(closing['IND_CD1'].value_counts().index)
print(f' closing데이터에서 10개이하로 갖는 업종코드 : {len(tmp)}')


68129.0    1204
68112.0     777
68121.0     450
30391.0     441
68122.0     437
           ... 
13222.0       1
64200.0       1
63110.0       1
96112.0       1
25995.0       1
Name: IND_CD1, Length: 1360, dtype: int64
58222.0    175
41229.0     98
46311.0     89
68112.0     70
68129.0     67
          ... 
29230.0      1
47812.0      1
91113.0      1
17900.0      1
23900.0      1
Name: IND_CD1, Length: 1037, dtype: int64

=================널값확인====================

11
150


================10개 이하로 갖고있는 업종코드 확인===============
 active데이터에서 10개이하로 갖는 업종코드 : 821
 closing데이터에서 10개이하로 갖는 업종코드 : 893


In [ ]:
# 공공기관 유형
# 값의 차이가 너무 미세하다..
# drop 결정
print(active['PB_ORG_TYP'].value_counts())
print(closing['PB_ORG_TYP'].value_counts())

0    28947
3       29
2        6
Name: PB_ORG_TYP, dtype: int64
0    6739
Name: PB_ORG_TYP, dtype: int64


In [ ]:
# 중견기업보호여부
# 중견기업보호여부 일단 넣자. 막 유의미해보이진 않는데 그래도
print(active['MDSCO_PRTC_YN'].value_counts())
print(closing['MDSCO_PRTC_YN'].value_counts())

N    28884
Y       98
Name: MDSCO_PRTC_YN, dtype: int64
N    6728
Y      11
Name: MDSCO_PRTC_YN, dtype: int64


In [ ]:
# 본점기업코드는 value_counts의 개수를 대신 넣어주면 좋을 것같다. 숫자가 높을수록 체인점을 많이 보유한 업종임을 판단할 수 있다.
print(active['HDOF_CMP_CD'].value_counts())
print(closing['HDOF_CMP_CD'].value_counts())

1308524.0    217
1365262.0     41
1857086.0     36
1493057.0     28
1713224.0     25
            ... 
1214685.0      1
1804072.0      1
1495765.0      1
1587373.0      1
1583116.0      1
Name: HDOF_CMP_CD, Length: 1616, dtype: int64
1387023.0    21
2709065.0    17
1741554.0    15
3215669.0    13
3741679.0    13
             ..
1101802.0     1
3173218.0     1
2872705.0     1
1146450.0     1
9189675.0     1
Name: HDOF_CMP_CD, Length: 1028, dtype: int64


In [ ]:
# 설립구분은 일단 그냥 넣자. 널값은 없다.
# 타입만 int로 바꿔주자
print(active['ESTB_GB'].value_counts())
print(closing['ESTB_GB'].value_counts())




# 널값 없다.
print(active['ESTB_GB'].isna().sum())
print(closing['ESTB_GB'].isna().sum())

1     28463
4       389
99      122
6         4
2         1
10        1
5         1
7         1
Name: ESTB_GB, dtype: int64
1     6446
4      128
99      99
5       37
2       22
7        7
Name: ESTB_GB, dtype: int64
0
0


In [ ]:
# 상장일자 - 없는게 많다.
# 있으면 1로 라벨링 없으면 0으로 라벨링
print(active['LIST_DATE'].isna().sum())
print(closing['LIST_DATE'].isna().sum())

In [ ]:
# 국가명
# 나중에 국가별로 경제상황이나 중소기업 지원정책이나 끌어올 수 있으면 좋을듯
# 일단 널값 etc로 채워서 넣는다.
print(active['NATN_NM'].value_counts())
print(closing['NATN_NM'].value_counts())

일본        379
미국        276
싱가포르      146
네덜란드      104
홍콩        103
         ... 
도미니카연방      1
방글라데시       1
폴란드         1
우즈베키스탄      1
라트비아        1
Name: NATN_NM, Length: 63, dtype: int64
일본        41
싱가포르      15
미국        15
홍콩        12
중국         8
독일         7
네덜란드       7
케이만군도      6
버진아일랜드     4
영국         4
스위스        4
몰타         3
대만         2
룩셈부르크      2
캐나다        2
브루나이       1
필리핀        1
기타(구주)     1
벨기에        1
말레이시아      1
오스트리아      1
인도네시아      1
덴마크        1
노르웨이       1
이스라엘       1
호주         1
인도         1
Name: NATN_NM, dtype: int64


In [ ]:
# 대표자명 컬럼의 널값은 각각 한개씩 갖고 있다.
# 그래서 그냥 널값 1개씩은 'etc'로 채워주고 출력
print(active['CEO_NM'].isna().sum())
print(closing['CEO_NM'].isna().sum())

1
1


In [ ]:
active.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28982 entries, 0 to 28981
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   BIZ_NO           28982 non-null  int64  
 1   CMP_NM           28982 non-null  object 
 2   BZ_TYP           28982 non-null  object 
 3   CMP_SCL          28982 non-null  int64  
 4   PBCO_GB          28982 non-null  int64  
 5   PSN_CORP_GB      28982 non-null  int64  
 6   HDOF_BR_GB       28982 non-null  int64  
 7   FR_IVST_CORP_YN  28982 non-null  object 
 8   VENT_YN          28982 non-null  object 
 9   LIST_CD          89 non-null     object 
 10  IND_CD_ORDR      28972 non-null  float64
 11  IND_CD1          28971 non-null  float64
 12  IND_CD2          1200 non-null   float64
 13  IND_CD3          263 non-null    float64
 14  PB_ORG_TYP       28982 non-null  int64  
 15  MDSCO_PRTC_YN    28982 non-null  object 
 16  HDOF_CMP_CD      2907 non-null   float64
 17  ESTB_DATE   

# 논의해볼 사항
- 산업코드1 처리하는거 진짜 애매하다